## Milestone 2
Dataset: Open Food Facts

The dataset is downloaded and stored in the /data folder

When describing the data, in particular, you should show (non-exhaustive list):

- That you can handle the data in its size.
- That you understand what’s into the data (formats, distributions, missing values, correlations, etc.).
- That you considered ways to enrich, filter, transform the data according to your needs.
- That you have updated your plan in a reasonable way, reflecting your improved knowledge after data acquaintance. In particular, discuss how your data suits your project needs and discuss the methods you’re going to use, giving their essential mathematical details in the notebook.
- That your plan for analysis and communication is now reasonable and sound, potentially discussing alternatives to your choices that you considered but dropped.


In [1]:
import pandas as pd
import numpy as np
import scipy as sp

'''
import findspark
findspark.init()

from pyspark.sql import *
from pyspark.sql import functions as F
from pyspark import SparkContext

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext''';

In [2]:
data_folder = './data/'

# Loading the data

## Open Food Facts dataset

The data is in the CSV file format and can be downloaded on the Openfoodfacts website. Its size is 1.6 GB and we first decided for this milestone to download and load it using spark, as we were not sure if Pandas could handle a file of this size but as we quickly realized it was working smoothly so we will use Pandas to manipulate the data.

In [3]:
#data = spark.read.option("delimiter", "\t").option("header", "true").csv(data_folder + "en.openfoodfacts.org.products.csv")
data = pd.read_csv(data_folder + 'en.openfoodfacts.org.products.csv', sep='\t', encoding='utf-8', low_memory=False)

In [6]:
data.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1529059204,2018-06-15T10:40:04Z,Vitória crackers,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000031,http://world-en.openfoodfacts.org/product/0000...,isagoofy,1539464774,2018-10-13T21:06:14Z,1539464817,2018-10-13T21:06:57Z,Cacao,NaN,130 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000000000123,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1535737982,2018-08-31T17:53:02Z,1535737986,2018-08-31T17:53:06Z,Sauce Sweety chili 0%,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000000000178,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1542456332,2018-11-17T12:05:32Z,1542456333,2018-11-17T12:05:33Z,Mini coco,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000000000291,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1534239669,2018-08-14T09:41:09Z,1534239732,2018-08-14T09:42:12Z,Mendiants,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We see that we have a lot of columns (173), not all of them will be useful for our project so we will select the most interesting ones and drop all the others to avoid keeping unused data for computations. We first chose to remove the columns related to subjects not related to our project for better readability like ingredients concentration per product, images, related to palm_oil...

In [45]:
%pprint
not_related = ['100g', 'image', 'palm_oil', 'code', 'url']
columns = [column for column in list(data.columns) if not any(st in column for st in not_related)] 
columns

Pretty printing has been turned OFF


['creator', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime', 'product_name', 'generic_name', 'quantity', 'packaging', 'packaging_tags', 'brands', 'brands_tags', 'categories', 'categories_tags', 'categories_en', 'origins', 'origins_tags', 'manufacturing_places', 'manufacturing_places_tags', 'labels', 'labels_tags', 'labels_en', 'cities', 'cities_tags', 'purchase_places', 'stores', 'countries', 'countries_tags', 'countries_en', 'ingredients_text', 'allergens', 'allergens_en', 'traces', 'traces_tags', 'traces_en', 'serving_size', 'serving_quantity', 'no_nutriments', 'additives_n', 'additives', 'additives_tags', 'additives_en', 'nutrition_grade_uk', 'nutrition_grade_fr', 'pnns_groups_1', 'pnns_groups_2', 'states', 'states_tags', 'states_en', 'main_category', 'main_category_en']

From the remainings columns we chose the ones that interests us the most. We made our choice first by judging the usefulness of each column in relation to our project and then by regarding in more details if the data in the column was usable.

In [65]:
%pprint
#TODO supprimez tous les elements de la liste qui peuvent etre inutiles
#ON DROP pas encore les rows NaNs comme on sait pas encore quelles values on va utiliser
keep = ['product_name', 'categories','categories_tags','categories_en','origins','origins_tags','manufacturing_places','manufacturing_places_tags','labels','labels_tags','labels_en','cities_tags','countries','countries_tags','countries_en','allergens','nutrition_grade_fr','states','states_tags','states_en','main_category','main_category_en']
selected_data = data[keep]
selected_data.head()

Pretty printing has been turned ON


,product_name,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,...,countries,countries_tags,countries_en,allergens,nutrition_grade_fr,states,states_tags,states_en,main_category,main_category_en
0,Vitória crackers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,en:france,en:france,France,NaN,NaN,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN
1,Cacao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,en:FR,en:france,France,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN
2,Sauce Sweety chili 0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,en:france,en:france,France,NaN,NaN,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN
3,Mini coco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,en:france,en:france,France,NaN,NaN,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN
4,Mendiants,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,en:france,en:france,France,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN


We save this dataframe in a csv file to speed up the loading process in our next runs.

In [59]:
selected_data.to_csv(data_folder + 'selected_data.csv')
#data = pd.read_csv(data_folder + 'en.openfoodfacts.org.products.csv', sep='\t', encoding='utf-8', low_memory=False)

Since there seems to be a lot of null values in the dataset, it can be interesting to take a look at the number of values that we have per remaining selected column.

In [60]:
selected_data.count()

product_name                 672399
categories                   180593
categories_tags              180592
categories_en                180558
origins                       42658
origins_tags                  42604
manufacturing_places          67510
manufacturing_places_tags     67490
labels                       102216
labels_tags                  102249
labels_en                    102249
cities_tags                   29723
countries                    697880
countries_tags               697880
countries_en                 697880
allergens                     69125
nutrition_grade_fr           141605
states                       698409
states_tags                  698409
states_en                    698409
main_category                180481
main_category_en             180481
dtype: int64

- Which countries are the highest exporters and importers and is there a relationship with the GDP?

For this one, we're interested in the following tags: `origin`, `manufacturing_places` and `countries`

In [7]:
#TODO Supprimer cette case et écrire plutot analyse des value au dessus ?
print('There are {} products that contains the origins tags'.format(data.filter(data.origins != "").count()))
print('There are {} products that contains the manufacturing_places tags'.format(data.filter(data.manufacturing_places != "").count()))
print('There are {} products that contains the countries tags'.format(data.filter(data.countries != "").count()))

There are 42658 products that contains the origins tags
There are 67510 products that contains the manufacturing_places tags
There are 697880 products that contains the countries tags


Here, we are facing our first problem, while the countries have more than enough samples in it (we still need to check the distribution later on), the `origins` and `manufacturing_places` both represent less than 10% of the data. Now, if we check at the actual values of them:

In [69]:
selected_data.manufacturing_places.unique()[:100]

array([nan, 'France', 'Brossard Québec', 'United Kingdom',
       'Brossard,Québec', 'États-Unis', 'France,Avranches', 'Etats-unis',
       'Brossars,Québec', 'Thailand', 'Belgien', 'net wt',
       'Las Ventas de Retamosa,Toledo (provincia),Castilla-La Mancha,España',
       'Saint Yrieix,France', 'Germany',
       'France,Limousin,87500,Saint-Yrieux', 'France,87500',
       '87500,France', 'sarlat', 'Pays-Bas,Netherlands', 'Royaume-Uni',
       'Royaume-Uni,Ecosse', 'Royaume Uni', 'Belgique', 'Ireland',
       '87500 Saint Yrieix', 'Écosse,Royaume-Uni', 'Black Sheep Brewery',
       'ESTADOS UNIDOS AMERICANOS', 'Vietnam', 'Argentine', 'USA',
       '30800 st Gilles', 'Chester, United Kingdom', 'UK',
       'Royaume-Uni,Irelande', 'California,USA', 'Italie', 'China',
       'Angleterre', 'Switzerland', 'Canada', 'México',
       'The Hershey Company', 'Cincinnati', 'United States', 'TAIWAN',
       'Taiwan', 'Japon', 'Japan', 'topco',
       'San Nicolas de los Garza, Nuevo León', 'SA

In [70]:
selected_data.origins.unique()[:100]

array([nan, 'France', 'Québec', 'Québec,Canada', 'United Kingdom',
       'Germany', 'UE', 'Canada', 'México', 'Grande Bretagne',
       'ESTADOS UNIDOS AMERICANOS', 'Royaume-Uni', 'Vietnam', 'Argentine',
       '?', 'brazil', 'England', 'Europe,Royaume-Uni',
       'Easter Grangemuir Farm,Pittenweem,Fife,KY10 2RB,Scotland,United Kingdom',
       'Royaume-Uni,Hors Royaume-Uni', 'perou', 'Italie', 'tibet',
       'Espagne,Royaume-Uni', 'United States', 'Madagascar', 'TAIWAN',
       'Japon', 'USA', 'Estados Unidos', 'CALIFORNIA', 'Estero',
       'Atlantique Nord-Ouest,Canada', 'Estados unidos', 'Suisse',
       'Saudi', 'Estados Unidos de América', 'Britain,British Chicken',
       'Italy', 'Francia', 'Fougerolles,France', 'MEXICO', 'mexico',
       'Ancaster,Ontario,canada', 'Scotland', 'Royaume-Uni,West Sussex',
       'Malaisie', 'Washington,USA', 'ESTADOS UNIDOS', 'E.U.A',
       'Etats-Unis', 'Californie,Etats-Unis', 'New Zealand',
       'Sicile,Italie', 'indéterminé', 'États-Uni

In [105]:
a = selected_data['origins'].dropna(how='all') #TODO chercher en lowercase pour trouver aussi avec minuscules
a[a.str.contains('Suisse')]

11255                      Suisse
71519                Vevey,Suisse
73202                      Suisse
92403                      Suisse
92407                      Suisse
140912                     Suisse
143171                     Suisse
143829                     Suisse
171381                     Suisse
210945                     Suisse
211979                     Suisse
212034             Suisse,Dorenaz
212662                     Suisse
212894                     Suisse
213270                     Suisse
213273                     Suisse
221179                     Suisse
221251                     Suisse
223776                     Suisse
227011                     Suisse
228141    Suisse,Italie,Allemagne
230304                     Suisse
230514                     Suisse
230528                     Suisse
230530                     Suisse
230537                     Suisse
230544                     Suisse
230557                     Suisse
230597                     Suisse
230678        

In [103]:
#DEMO de comment on peut chercher les pays d'origine qu'on veut
a = selected_data['origins'].dropna(how='all') #TODO chercher en lowercase pour trouver aussi avec minuscules
a[a.str.contains('France')]

254                                                  France
272                                                  France
313                                                  France
362                                                  France
416                                                  France
597                                                  France
799                                                  France
1009                                                 France
1208                                                 France
11747                                    Fougerolles,France
14473                                                France
21855                                        Ardèche,France
21859                                        Ardèche,France
21870                                        Ardèche,France
21888                                                France
38256                                                France
44789                                   

We are now facing another problem, all the tags are not normalized and a lot of them are even invalid ("mer", postal code, or in other languages). 

## GDP and Life Expectancy

We found the GDP per country (in USD) on the World Bank website

In [7]:
gdp = spark.read.option("header", "true").csv(data_folder + 'GDP.csv').select('Country Name', 'Country Code', '2016').withColumnRenamed('Country Name', 'countries_en')

In [8]:
gdp.show()

+--------------------+------------+----------------+
|        countries_en|Country Code|            2016|
+--------------------+------------+----------------+
|               Aruba|         ABW|            null|
|         Afghanistan|         AFG|19469022207.6852|
|              Angola|         AGO|95337203468.1156|
|             Albania|         ALB|11883682170.8236|
|             Andorra|         AND|2877311946.90265|
|          Arab World|         ARB|2500164034395.78|
|United Arab Emirates|         ARE|357045064669.843|
|           Argentina|         ARG| 554860945013.62|
|             Armenia|         ARM| 10546135160.031|
|      American Samoa|         ASM|       658000000|
| Antigua and Barbuda|         ATG| 1460144703.7037|
|           Australia|         AUS|1208039015868.39|
|             Austria|         AUT|390799991147.468|
|          Azerbaijan|         AZE|37867518957.1975|
|             Burundi|         BDI| 3007029030.4001|
|             Belgium|         BEL|46754554876

Same goes for the life expectancy:

In [9]:
le = spark.read.option("header", "true").csv(data_folder + 'LE.csv').select('Country Name', 'Country Code', '2016').withColumnRenamed('Country Name', 'countries_en')

In [10]:
le.dtypes

[('countries_en', 'string'), ('Country Code', 'string'), ('2016', 'string')]

In [11]:
le.show(10)

+--------------------+------------+---------------+
|        countries_en|Country Code|           2016|
+--------------------+------------+---------------+
|               Aruba|         ABW|         75.867|
|         Afghanistan|         AFG|         63.673|
|              Angola|         AGO|         61.547|
|             Albania|         ALB|         78.345|
|             Andorra|         AND|           null|
|          Arab World|         ARB|71.198456370659|
|United Arab Emirates|         ARE|         77.256|
|           Argentina|         ARG|         76.577|
|             Armenia|         ARM|         74.618|
|      American Samoa|         ASM|           null|
+--------------------+------------+---------------+
only showing top 10 rows



# Cleaning the data

## Open Food Facts dataset

## Countries

In [12]:
data_countries = data.filter(data.countries_en != "")

In [13]:
col_split = F.split(data_countries['countries_en'], ',')

In [14]:
data_countries = data_countries.withColumn('countries_en', F.explode(col_split))

In [15]:
data_countries.select('countries_en').distinct().show(500)

+--------------------+
|        countries_en|
+--------------------+
|       Côte d'Ivoire|
|                Chad|
|            Anguilla|
|              Russia|
|            Paraguay|
|Virgin Islands of...|
|               World|
|               Yemen|
|British Indian Oc...|
|             Senegal|
|              Sweden|
|              Guyana|
|         Philippines|
|            Djibouti|
|           Singapore|
|            Malaysia|
|fr:republica-moldova|
|        ch:allemagne|
|                Fiji|
|              Turkey|
|           fr:nantes|
|Nutrition facts c...|
|              Malawi|
|                Iraq|
|           fr:tahiti|
|             Germany|
|                  En|
|            Cambodia|
|     To be completed|
|         Afghanistan|
|            de:grece|
|              Jordan|
|              Rwanda|
|            Maldives|
|    Photos validated|
|          ch:schweiz|
|              France|
|            de:japon|
|              Greece|
|     Photos uploaded|
|Packaging 

Some of the entires are still invalid because they are written in another languages, we decided to not count them. Since we already have a list of countries, we are going to use them to keep only the valid entries.

In [16]:
joined = data_countries.join(gdp, 'countries_en', how='inner').drop('Country Code', '2016')

In [26]:
joined.filter(data.origins != "").count()

45504

In [31]:
origins = joined.join(gdp, joined.origins.isin(gdp.countries_en), how='inner')

In [33]:
origins.count()

15203

In [34]:
or_pd = origins.toPandas()

In [ ]:
or_pd.origins.hist()